# Data Loading & Cleaning

Load raw insurance data, perform initial exploration, and apply data cleaning rules.

In [1]:
import pandas as pd
import numpy as np
import os
import warnings
warnings.filterwarnings('ignore')

# Create output directories if they don't exist
os.makedirs('../results', exist_ok=True)
os.makedirs('../data/processed', exist_ok=True)

print("✓ Libraries imported and directories created")

✓ Libraries imported and directories created


## Load Raw Data

In [2]:
# Load raw data from data/raw directory
df_raw = pd.read_csv("../data/raw/freMTPL2freq.csv")

print("=" * 80)
print("RAW DATA LOADED FROM: ../data/raw/freMTPL2freq.csv")
print("=" * 80)
print(f"Dataset shape: {df_raw.shape}")
print(f"\nColumns: {list(df_raw.columns)}")
print(f"\nFirst few rows:")
print(df_raw.head())
print(f"\nData types:")
print(df_raw.dtypes)
print(f"\nBasic statistics:")
print(df_raw.describe())
print(f"\nMissing values:")
print(df_raw.isnull().sum())

RAW DATA LOADED FROM: ../data/raw/freMTPL2freq.csv
Dataset shape: (678013, 12)

Columns: ['IDpol', 'ClaimNb', 'Exposure', 'Area', 'VehPower', 'VehAge', 'DrivAge', 'BonusMalus', 'VehBrand', 'VehGas', 'Density', 'Region']

First few rows:
   IDpol  ClaimNb  Exposure Area  VehPower  VehAge  DrivAge  BonusMalus  \
0    1.0        1      0.10    D         5       0       55          50   
1    3.0        1      0.77    D         5       0       55          50   
2    5.0        1      0.75    B         6       2       52          50   
3   10.0        1      0.09    B         7       0       46          50   
4   11.0        1      0.84    B         7       0       46          50   

  VehBrand   VehGas  Density Region  
0      B12  Regular     1217    R82  
1      B12  Regular     1217    R82  
2      B12   Diesel       54    R22  
3      B12   Diesel       76    R72  
4      B12   Diesel       76    R72  

Data types:
IDpol         float64
ClaimNb         int64
Exposure      float64
Area 

## Data Cleaning & Processing

In [3]:
# Data cleaning and preprocessing
df = df_raw.copy()

# Define variables: Remove records with invalid exposure and claims
valid_exposure_threshold = 0
valid_claims_min = 0

# Apply cleaning rules
df = df[(df["Exposure"] > valid_exposure_threshold) & (df["ClaimNb"] >= valid_claims_min)].copy()

n_removed = len(df_raw) - len(df)
print("=" * 80)
print("DATA CLEANING & PROCESSING SUMMARY")
print("=" * 80)
print(f"Original records: {len(df_raw):,}")
print(f"Removed records: {n_removed:,}")
print(f"Cleaned records: {len(df):,}")

# Core feature: Calculate claim frequency
df["freq"] = df["ClaimNb"] / df["Exposure"]

print(f"\nClaim Frequency Statistics:")
print(df["freq"].describe())

# Define variables for feature engineering
overdispersion_ratio = df['ClaimNb'].var() / df['ClaimNb'].mean()
print(f"\nOverdispersion Ratio (Variance/Mean): {overdispersion_ratio:.3f}")

DATA CLEANING & PROCESSING SUMMARY
Original records: 678,013
Removed records: 0
Cleaned records: 678,013

Claim Frequency Statistics:
count    678013.000000
mean          0.263964
std           4.593915
min           0.000000
25%           0.000000
50%           0.000000
75%           0.000000
max         732.000000
Name: freq, dtype: float64

Overdispersion Ratio (Variance/Mean): 1.083


## Feature Engineering - Create Age Groups

In [4]:
# Feature Engineering: Create age bins with defined boundaries
driver_age_bins = [18, 25, 35, 50, 70, 100]
driver_age_labels = ['18-25', '25-35', '35-50', '50-70', '70+']
df["DrivAge_group"] = pd.cut(df["DrivAge"], bins=driver_age_bins, labels=driver_age_labels, include_lowest=True)

vehicle_age_bins = [0, 1, 5, 10, 20, 100]
vehicle_age_labels = ['0-1Y', '1-5Y', '5-10Y', '10-20Y', '20+Y']
df["VehAge_group"] = pd.cut(df["VehAge"], bins=vehicle_age_bins, labels=vehicle_age_labels, include_lowest=True)

print("\n--- Driver Age Groups ---")
driver_age_freq = df.groupby("DrivAge_group", observed=True)["freq"].agg(['mean', 'count', 'std'])
print(driver_age_freq)

print("\n--- Vehicle Age Groups ---")
vehicle_age_freq = df.groupby("VehAge_group", observed=True)["freq"].agg(['mean', 'count', 'std'])
print(vehicle_age_freq)

print("\n--- Vehicle Power (Top 10) ---")
vehi_power_freq = df.groupby("VehPower", observed=True)["freq"].agg(['mean', 'count', 'std']).head(10)
print(vehi_power_freq)


--- Driver Age Groups ---
                   mean   count       std
DrivAge_group                            
18-25          0.404561   38895  4.737294
25-35          0.245015  149183  4.322840
35-50          0.265029  251693  4.987477
50-70          0.256378  198877  4.393094
70+            0.228376   39365  3.732235

--- Vehicle Age Groups ---
                  mean   count       std
VehAge_group                            
0-1Y          0.612589  129023  8.101648
1-5Y          0.175993  191614  3.382571
5-10Y         0.213167  171594  3.720776
10-20Y        0.161794  177460  2.603155
20+Y          0.110563    8322  2.453202

--- Vehicle Power (Top 10) ---
              mean   count       std
VehPower                            
4         0.276456  115349  4.678317
5         0.299052  124821  4.743427
6         0.255648  148976  4.896477
7         0.247424  145401  4.307366
8         0.146438   46956  2.718668
9         0.330009   30085  4.540396
10        0.331983   31354  5.299763

## Save Cleaned Data

In [5]:
# Save processed data
output_file = '../data/processed/processed.csv'
df.to_csv(output_file, index=False)
print(f"✓ Processed data SAVED to: {output_file}")

# Display summary
print(f"\nProcessed dataset shape: {df.shape}")
print(f"Columns: {list(df.columns)}")

✓ Processed data SAVED to: ../data/processed/processed.csv

Processed dataset shape: (678013, 15)
Columns: ['IDpol', 'ClaimNb', 'Exposure', 'Area', 'VehPower', 'VehAge', 'DrivAge', 'BonusMalus', 'VehBrand', 'VehGas', 'Density', 'Region', 'freq', 'DrivAge_group', 'VehAge_group']
